# Inicializar e importar pacotes

In [1]:
import geopandas as gpd
import pandas as pd
import math
import ee
import geemap
import json

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import image collections

Landsat 7 - PY6S

In [ ]:
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/Landsat7")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print(ee.Date(landsat7.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2000-01-24T13:46:48
2024-01-16T11:16:49


In [ ]:
print(landsat7.aggregate_histogram('WRS_PATH').getInfo())
print(landsat7.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 624, '228.0': 729, '229.0': 13}
{'61.0': 685, '62.0': 681}


In [ ]:
print(landsat7.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat7.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 298, '62.0': 326}
{'61.0': 380, '62.0': 349}


Landsat 8 PY6S

In [ ]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/Landsat8")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat8.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-11T13:55:54
2024-12-12T13:47:48


In [ ]:
print(landsat8.aggregate_histogram('WRS_PATH').getInfo())
print(landsat8.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 364, '228.0': 379}
{'61.0': 372, '62.0': 371}


In [ ]:
print(landsat8.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat8.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 180, '62.0': 184}
{'61.0': 192, '62.0': 187}


Landsat 9 PY6S

In [ ]:
landsat9 = (ee.ImageCollection("projects/ee-curuai2/assets/Py6S/LD9/Landsat9")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat9.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat9.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2021-11-06T13:49:55
2024-12-27T13:54:34


In [ ]:
print(landsat9.aggregate_histogram('WRS_PATH').getInfo())
print(landsat9.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 101, '228.0': 105, '229.0': 2}
{'61.0': 104, '62.0': 104}


In [ ]:
print(landsat9.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat9.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 50, '62.0': 51}
{'61.0': 53, '62.0': 52}


# Transform into remote sensing reflectance and sunglint correction

In [ ]:
def deglint (img):
  '''#dividir imagem corrigida por pi
  Rrs_sat_ac = Rsat_ac / pi
  fazer deglint
  Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
  correção no artigo INPE CURUAI'''
  Rrs = img.divide(math.pi)
  deglint = Rrs.select(['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2'])\
    .subtract(Rrs.select('swir1'))

  return (deglint
          .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

## Padronize band names

In [ ]:
name_bands = ['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2']

Landsat 7

In [ ]:
#renomear bandas

ld7 = landsat7.map(lambda img: img.rename(name_bands))
display(ld7.size().getInfo())

1366

Landsat 8

In [ ]:
ld8 = landsat8.map(lambda img: img.rename(name_bands))
display(ld8.size().getInfo())

743

landsat 9

In [ ]:
ld9 = landsat9.map(lambda img: img.rename(name_bands))
display(ld9.size().getInfo())

208

In [ ]:
merge_col = ld8.merge(ld7).merge(ld9).sort('system:time_start').map(deglint)
merge_col.limit(5)

In [ ]:
merge_col.size()

# Importar dados de periodo baseado na vazao de obidos

In [ ]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])
# df_period_limits['Data'] = pd.to_datetime(df_period_limits['Data'])
df_period_limits


,Data,type,day,month,year,longitude,latitude
0,2000-01-13 10:43:38.181818240,LW_to_R,13,1,2000,-55.5131,-1.9192
1,2000-02-29 18:17:42.295081984,R_to_HW,29,2,2000,-55.5131,-1.9192
2,2000-08-24 20:45:42.857142784,HW_to_F,24,8,2000,-55.5131,-1.9192
3,2000-09-28 10:23:44.598930432,F_to_LW,28,9,2000,-55.5131,-1.9192
4,2001-01-05 16:27:54.418604672,LW_to_R,5,1,2001,-55.5131,-1.9192
...,...,...,...,...,...,...,...
95,2023-08-25 18:37:31.737451520,F_to_LW,25,8,2023,-55.5131,-1.9192
96,2024-01-26 18:57:28.739495680,LW_to_R,26,1,2024,-55.5131,-1.9192
97,2024-03-25 23:32:02.330097152,R_to_HW,25,3,2024,-55.5131,-1.9192
98,2024-04-06 10:42:51.428571648,HW_to_F,6,4,2024,-55.5131,-1.9192


# Fazer mosaico por periodo - definido por meses

In [ ]:
years = ee.List.sequence(2000,2024);
months = ee.List.sequence(1,12,3);
display(months.getInfo())

[1, 4, 7, 10]

In [ ]:
monthly_mosaics =  ee.ImageCollection.fromImages(
    years.map(lambda y: months.map(lambda m: merge_col.filter(ee.Filter.calendarRange(y, y, 'year'))
    .filter(ee.Filter.calendarRange(m, ee.Number(m).add(2), 'month')).median()
    .set('year', y).set('month_init', m).set('month_end', ee.Number(m).add(2)).set('system:time_start', ee.Date.fromYMD(y, m, 1)).set('timestamp', ee.Date.fromYMD(y, m, 1).format('yyyy-MM-dd'))
    )).flatten());
display(monthly_mosaics.size().getInfo())

100

In [ ]:
monthly_mosaic = monthly_mosaics.map(lambda img: img.set('band_count',img.bandNames().length())).filter(ee.Filter.gt('band_count',0))
display(monthly_mosaic.size().getInfo())
display(monthly_mosaic.limit(2).getInfo())

100

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'blue_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'green_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'red_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'nir_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'swir1',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'swir2',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0

# Fazer mosaico definido por vazao

In [ ]:
list_images = []
for i in range(0,len(df_period_limits)-1):
  # print(df_period_limits['Data'][i][0:10])
  # print(df_period_limits['Data'][i+1][0:10])

  image = (merge_col.filter(ee.Filter.date(df_period_limits['Data'][i][0:10],df_period_limits['Data'][i+1][0:10]))
              .median())
  if int(image.bandNames().length().getInfo()) > 0:
    list_images.append(image
              .set('year', str(df_period_limits['year'][i]))
              .set('month_init', str(df_period_limits['month'][i]))
              .set('month_end', str(df_period_limits['month'][i+1]))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str(df_period_limits['Data'][i][0:10])))
              .set('time_start', str(df_period_limits['Data'][i][0:10]))
              .set('time_finish', str(df_period_limits['Data'][i+1][0:10]))
              .set('band_count',image.bandNames().length()))



len(list_images)

98

In [ ]:
list_images.append(merge_col.filterDate('2024-05-30','2025-01-01').median()
              .set('year', str(2024))
              .set('month_init', str('05'))
              .set('month_end', str(12))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str('2024-05-30')))
              .set('time_start', str('2024-05-30'))
              .set('time_finish', str('2025-01-01'))
              .set('band_count',image.bandNames().length()))



In [ ]:
period_mosaics =  ee.ImageCollection.fromImages(ee.List(list_images))
period_mosaics.size()

# Calculate area

In [ ]:
# Mask land function
def hsvComposite (image):
    composite = image.select(['blue_mean','green_mean','red_mean']).rgbToHsv()#.clip(limits);
    hue = composite.select("hue");
    max_mask = hue.lte(0.9)
    min_mask = hue.gte(0.3)
    return image.updateMask(max_mask).updateMask(min_mask).select(['blue_mean','green_mean','red_mean','nir_mean']);

In [ ]:
def area_calc(img):
  '''receives an image of curuai and returns water surface area in km2 within the floodplain limits
  as a property of the input image'''

  # Get a pixel area image.
  pixel_area = ee.Image.pixelArea()

  floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/varzea_alagavel')
  image = hsvComposite(img)
  img_mask = image.gt(0)

  areaImage = img_mask.multiply(pixel_area)

  area = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': floodplain.geometry(),
    'scale': 30,
    'maxPixels': 1e10
    })
  return image.set('area_km2',ee.Number(area.get('red_mean')).divide(1e6))


## monthly

In [ ]:
monthly_area = monthly_mosaic.map(area_calc)
monthly_area.first()

## by discharge

In [ ]:
period_area = period_mosaics.map(area_calc)

period_area.limit(3)

# Import Model and Classify Images

In [ ]:
classifier = ee.Classifier.load('projects/ee-curuai2/assets/RF_GEE_regressor')
print(classifier.getInfo())

{'type': 'Classifier.load', 'id': 'projects/ee-curuai2/assets/RF_GEE_regressor'}


In [ ]:
predictors = ['blue_mean', 'green_mean','red_mean', 'nir_mean']

## by month

In [ ]:
spm_classified = monthly_area.select(predictors).map(lambda img: img.addBands(img.classify(classifier=classifier)))

display(spm_classified.size().getInfo())

100

98

In [ ]:
spm_classified.limit(5)

## by discharge

In [ ]:
spm_period_classified = period_area.select(predictors).map(lambda img: img.addBands(img.classify(classifier=classifier)))

display(spm_period_classified.size().getInfo())

99

In [ ]:
spm_period_classified.limit(5)

# Export as Asset: Image Collection

In [ ]:
prj = landsat8.first().select('B4').projection().getInfo()
scale = landsat8.first().select('B4').projection().nominalScale().getInfo()
region = ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds()

## by month

In [ ]:
def export_img(img):
      # define YOUR assetID
    # export

    fname = ee.String(img.get('timestamp')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(img),
        description= 'ld_mosaic_'+fname,
        assetId = 'projects/ee-curuai2/assets/landsat_water_period/water_period/mosaic_'+fname,#change the properties to add in images here
        region = ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
        crs = prj['crs'],
        scale = scale,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return img

In [ ]:
col_length = spm_classified.size().getInfo()
#print(col_length)
# cannot map the function because we run things in the function in bothe client and server sides so we need to make a loop
# for very big time series It is recomendable to break the series and export data in parts
# not only because of the loop but also because of how exporting to assets works in google earth engine
# and you can run into problems if too much information is exported at the same time
for i in range(0,col_length):
    #print(i)
    list = spm_classified.toList(col_length)
    img = ee.Image(list.get(i))
    export_img(img)

## by discharge

In [ ]:
def export_img(img):
      # define YOUR assetID
    # export

    fname = ee.String(img.get('time_start')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(img),
        description= 'ld_mosaic_'+fname,
        assetId = 'projects/ee-curuai2/assets/landsat_water_period/water_period_discharge/mosaic_'+fname,#change the properties to add in images here
        region = region,
        crs = prj['crs'],
        scale = scale,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')

    # return img



In [ ]:
# geemap.ee_export_image_collection_to_asset(spm_period_classified, scale=30,crs='EPSG:32621',maxPixels=1e13,)

Total number of images: 98

Name "projects/ee-curuai2/assets/Py6S//" is invalid. Each segment must contain only the following characters: a..z, A..Z, 0..9, "_" or "-". Each segment must be at least 1 character long and at most 100 characters long.


In [ ]:
n = 5
col = spm_period_classified.filterDate('2024-04-07','2025-01-01').limit(n)
export_img(col.first())
col_length = col.size().getInfo()
# print(col_length)
# cannot map the function because we run things in the function in bothe client and server sides so we need to make a loop
# for very big time series It is recomendable to break the series and export data in parts
# not only because of the loop but also because of how exporting to assets works in google earth engine
# and you can run into problems if too much information is exported at the same time
lista = col.toList(n)
for i in range(0,col_length):
    # print(i)
    # lista = x.toList(50)
    img = ee.Image(lista.get(i))
    # print(img.getInfo())
    export_img(img)

exporting 2024-05-30--->done


EEException: User memory limit exceeded.

# Visualize gif

In [34]:
# import collection to generate gif
colecao = ee.ImageCollection('projects/ee-curuai2/assets/landsat_water_period/water_period_discharge')
colecao.first().bandNames()

In [27]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 700,
    "region": ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry(),
    "framesPerSecond": 5,
    "bands": ["classification"],
    "min": 1,
    "max": 350,
    "palette": ['blue','green', 'yellow', 'orange','red'],
}

In [28]:
# geemap.download_ee_video(spm_classified, video_args, '/content/classification_SPM.gif')
geemap.download_ee_video(colecao, video_args, 'classification_SPM_dis.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /content/classification_SPM_dis.gif


In [29]:
geemap.add_text_to_gif(
    '/content/classification_SPM_dis.gif',
    '/content/classification_SPM_text_dis.gif',
    xy=("3%", "5%"),
    text_sequence=colecao.aggregate_array('time_start').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)

In [30]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 700,
    "region": ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
    "framesPerSecond": 5,
    "bands": ["red_mean",'green_mean','blue_mean'],
    "min": 0,
    "max": 0.07,
}

In [36]:
geemap.download_ee_video(colecao, video_args, '/content/water_period.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /content/water_period.gif


In [37]:
geemap.add_text_to_gif(
    '/content/water_period.gif',
    '/content/water_period_text.gif',
    xy=("3%", "5%"),
    text_sequence=colecao.aggregate_array('time_start').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)